1. Класс для извлечения ключевых слов


Извлекаем ключевые слова с помощью библиотеки RAKE, который подходит для выделения основных идей и тем текста.

**RAKE (Rapid Automatic Keyword Extraction)** — это алгоритм для извлечения ключевых слов из текста с помощью анализа частот слов и их взаимосвяей. Слова и фразы, которые часто встречаются вместе и редко встречаются в других контекстах текста **считаются ключевыми**.



In [37]:
from rake_nltk import Rake
from nltk.corpus import stopwords
import nltk

# Предварительная подготовка
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pavelkockin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pavelkockin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [38]:
class RakeKeyphrasesExtractor:
    def __init__(self, language="russian", max_length=10):
        self.language = language
        self.max_length = max_length
        self.stopwords = stopwords.words(self.language)
        self.rake = Rake(
            stopwords=self.stopwords, language=self.language, max_length=self.max_length
        )

    def extract(self, text):
        self.rake.extract_keywords_from_text(text)
        keyphrases = self.rake.get_ranked_phrases_with_scores()
        return keyphrases


2. Класс для нормализации текста


In [39]:
import pymorphy2
from collections import Counter


class TextNormalizer:
    def __init__(self, language="ru"):
        self.morph = pymorphy2.MorphAnalyzer(lang=language)

    def normalize(self, keyphrases):
        filtered_tokens = []
        for _, text in keyphrases:
            for word in text.split():
                p = self.morph.parse(str(word))[0]
                if p.tag.POS == "NOUN":
                    filtered_tokens.append(p.normal_form)
        return Counter(filtered_tokens)


3. Класс для анализа на основе векторов слов


In [40]:
import fasttext
import fasttext.util
from sklearn.metrics import pairwise_distances
import numpy as np


class WordVectorAnalyzer:
    def __init__(self, language="ru", model_name="cc.ru.300.bin"):
        fasttext.util.download_model(language, if_exists="ignore")
        self.ft_model = fasttext.load_model(model_name)

    def analyze(self, words, top_n=5):
        wordlist_vec = [self.ft_model[word] for word in words]
        distances = pairwise_distances(wordlist_vec, wordlist_vec, "cosine")
        keywords_with_distance = list(zip(words, distances.mean(axis=1)))
        keywords_with_distance = sorted(keywords_with_distance, key=lambda x: -x[1])
        return [keyword for keyword, _ in keywords_with_distance][:top_n]


4. Всё запускаем

In [41]:
class KeywordExtractor:
    def __init__(self, language):
        self.language = language
        self.extractor = RakeKeyphrasesExtractor(language=language)
        self.normalizer = TextNormalizer(language="ru")
        self.analyzer = WordVectorAnalyzer(language="ru")

    def extract(self, text: str, top_n: int, min_keyword_cnt: int = 2):
        print("Экстрактим кейворды...")
        keyphrases = self.extractor.extract(text)
        print("Нормализуем и фильтруем кейворды...")
        normalized_words = self.normalizer.normalize(keyphrases)
        most_common_words = [
            word for word, cnt in normalized_words.most_common(top_n) if cnt >= min_keyword_cnt
        ]
        print("Анализируем кейворды для хэштэга...")
        keywords = self.analyzer.analyze(most_common_words, top_n=top_n)
        return keywords


In [46]:
class KeywordExtractorApplication:
    def __init__(self, language="russian"):
        self.language = language
        self.extractor = KeywordExtractor(self.language)

    def get_keywords(self, text, top_n):
        return self.extractor.extract(text, top_n)


app = KeywordExtractorApplication()

In [47]:
source_text = """
Методы ускорения инференса. 
Дистилляция
Обычно в этом методе есть какая-то большая модель, которую мы называем учитель (teacher), и модель поменьше — студент (student). Хорошим примером будет YandexGPT 3 — большая LLM, способная решать задачу с наилучшим качеством, но она совершенно не укладывается в наш вычислительный бюджет. Есть модель поменьше, вроде Т5, которая потребляет сильно меньше ресурсов, но не решает задачу так же качественно, как YandexGPT 3. Задача Knowledge Distillation состоит в том, чтобы минимизировать потери (loss) между фичами — предсказаниями учителя и студента. 
"""

hashtags = app.get_keywords(source_text, top_n=10)
print(hashtags)

Экстрактим кейворды...
Нормализуем и фильтруем кейворды...
Анализируем кейворды для хэштэга...
['модель', 'задача', 'метод', 'студент', 'учитель']
